In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Intellectual_Disability"
cohort = "GSE100680"

# Input paths
in_trait_dir = "../../input/GEO/Intellectual_Disability"
in_cohort_dir = "../../input/GEO/Intellectual_Disability/GSE100680"

# Output paths
out_data_file = "../../output/preprocess/Intellectual_Disability/GSE100680.csv"
out_gene_data_file = "../../output/preprocess/Intellectual_Disability/gene_data/GSE100680.csv"
out_clinical_data_file = "../../output/preprocess/Intellectual_Disability/clinical_data/GSE100680.csv"
json_path = "../../output/preprocess/Intellectual_Disability/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"APP levels mediate beta-amyloid but not tau-related aspects of Alzheimer’s disease in a Down syndrome model of cortical neurogenesis."
!Series_summary	"Early-onset Alzheimer’s disease-like pathology in Down syndrome (DS, trisomy 21) is commonly attributed to an increased dosage of the amyloid precursor protein (APP) gene.  To test this central tenet of the amyloid-cascade hypothesis we deleted the supernumerary copy of the APP gene in trisomic DS iPSC, or upregulated APP expression in euploid human pluripotent stem cell lines with dCas9-VP64, and subjected these lines to prolonged cortical neural differentiation. Our data reveal that increased APP gene dosage and expression is necessary and sufficient for increased beta-amyloid production and pyroglutamate(E3)-containing plaque deposition, but is neither sufficient nor required for tau hyperphosphorylation, neurofibrillary tangle formation, or increased oxidative stress-induced apoptosis in neuron

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From the background information, this dataset involves neuronal cultures from Down syndrome and euploid samples,
# which likely contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From analyzing the Sample Characteristics Dictionary:

# Trait (Intellectual Disability): The trait can be inferred from the description field (key 3)
# Down Syndrome (DS) vs Euploid samples
trait_row = 3

# Age: Age information is available in key 2
age_row = 2

# Gender: Not available in the data
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait value to binary (0=control, 1=case)"""
    if value is None or ':' not in value:
        return None
    
    value = value.split(':', 1)[1].strip()
    
    # DS (Down Syndrome) samples are cases (1), Euploid are controls (0)
    if 'DS ' in value:
        return 1
    elif 'Euploid' in value:
        return 0
    elif 'APP copy number-corrected DS' in value:
        # These are genetically modified DS samples where APP copy number is corrected
        # For the intellectual disability trait, they still have DS but with APP correction
        return 1
    else:
        return None

def convert_age(value):
    """Convert age value to continuous (days)"""
    if value is None or ':' not in value:
        return None
    
    value = value.split(':', 1)[1].strip()
    
    if 'Day ' in value:
        try:
            return int(value.replace('Day ', ''))
        except:
            return None
    else:
        return None

# No gender information available, but still define the function in case needed
def convert_gender(value):
    """Convert gender value to binary (0=female, 1=male)"""
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Save initial filtering results
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Use the library function to extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data, 
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical data
    print("Preview of clinical data:")
    print(preview_df(clinical_df))
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical data:
{'GSM2691139': [1.0, 45.0], 'GSM2691140': [1.0, 45.0], 'GSM2691141': [0.0, 45.0], 'GSM2691142': [0.0, 45.0], 'GSM2691143': [0.0, 45.0], 'GSM2691144': [1.0, 65.0], 'GSM2691145': [1.0, 65.0], 'GSM2691146': [1.0, 65.0], 'GSM2691147': [0.0, 65.0], 'GSM2691148': [0.0, 65.0], 'GSM2691149': [0.0, 65.0], 'GSM2691150': [1.0, 45.0], 'GSM2691151': [1.0, 45.0], 'GSM2691152': [0.0, 45.0], 'GSM2691153': [0.0, 45.0], 'GSM2691154': [0.0, 45.0], 'GSM2691155': [1.0, 65.0], 'GSM2691156': [1.0, 65.0], 'GSM2691157': [1.0, 65.0], 'GSM2691158': [0.0, 65.0], 'GSM2691159': [0.0, 65.0], 'GSM2691160': [0.0, 65.0], 'GSM2691161': [1.0, 45.0], 'GSM2691162': [1.0, 45.0], 'GSM2691163': [1.0, 45.0], 'GSM2691164': [0.0, 45.0], 'GSM2691165': [0.0, 45.0], 'GSM2691166': [0.0, 45.0], 'GSM2691167': [1.0, 65.0], 'GSM2691168': [1.0, 65.0], 'GSM2691169': [1.0, 65.0], 'GSM2691170': [0.0, 65.0], 'GSM2691171': [0.0, 65.0], 'GSM2691172': [0.0, 65.0]}
Clinical data saved to ../../output/preprocess/Intellec

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:


Successfully extracted gene data with 47323 rows
First 20 gene IDs:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers observed in the gene expression data are in the format "ILMN_XXXXXXX"
# These are Illumina probe IDs used in their microarray platforms, not human gene symbols
# Illumina probe IDs need to be mapped to human gene symbols for proper analysis

# Based on biomedical knowledge, these are Illumina BeadArray probe identifiers and not human gene symbols
# These identifiers will need to be mapped to official gene symbols for meaningful biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 1657123 rows

Gene annotation preview (first few rows):
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns contain the probe identifiers and gene symbols
print("Examining gene annotation columns for mapping...")

# The 'ID' column in gene_annotation contains the probe identifiers (ILMN_XXXXXXX)
# The 'Symbol' column contains the gene symbols
prob_col = 'ID'
gene_col = 'Symbol'

print(f"Using probe identifiers from column '{prob_col}' and gene symbols from column '{gene_col}'")

# 2. Get a gene mapping dataframe
print("Creating gene mapping dataframe...")
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Created mapping with {len(mapping_df)} probe-to-gene mappings")

# Preview the mapping dataframe
print("\nPreview of gene mapping data:")
print(mapping_df.head())

# Check if we have mappings for probes in our gene expression data
probes_in_expression = set(gene_data.index)
probes_in_mapping = set(mapping_df['ID'])
common_probes = probes_in_expression.intersection(probes_in_mapping)
print(f"\nNumber of probes in expression data: {len(probes_in_expression)}")
print(f"Number of probes in mapping data: {len(probes_in_mapping)}")
print(f"Number of common probes: {len(common_probes)}")
print(f"Percentage of expression probes with mapping: {len(common_probes)/len(probes_in_expression)*100:.2f}%")

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
print("\nConverting probe-level measurements to gene-level expression data...")
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Converted to gene expression data with {len(gene_data)} unique genes")

# Check for gene names that are null or empty strings
valid_genes = gene_data.index.notnull() & (gene_data.index != '')
gene_data = gene_data[valid_genes]
print(f"After filtering invalid gene names: {len(gene_data)} genes")

# Preview of the final gene expression data
print("\nPreview of gene expression data:")
print(gene_data.head())


Examining gene annotation columns for mapping...
Using probe identifiers from column 'ID' and gene symbols from column 'Symbol'
Creating gene mapping dataframe...
Created mapping with 44837 probe-to-gene mappings

Preview of gene mapping data:
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB

Number of probes in expression data: 47323
Number of probes in mapping data: 44837
Number of common probes: 44053
Percentage of expression probes with mapping: 93.09%

Converting probe-level measurements to gene-level expression data...
Converted to gene expression data with 21464 unique genes
After filtering invalid gene names: 21464 genes

Preview of gene expression data:
       GSM2691139  GSM2691140  GSM2691141  GSM2691142  GSM2691143  GSM2691144  \
Gene                                               

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols...")
try:
    gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization, gene data has {len(gene_data.index)} genes")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to: {out_gene_data_file}")
    is_gene_available = True
except Exception as e:
    print(f"Error during gene symbol normalization: {e}")
    is_gene_available = False

# 2. Load the clinical data and link with genetic data
print("\nLoading clinical data and linking with genetic data...")
try:
    # Load the clinical data
    clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
    print(f"Loaded clinical data with shape: {clinical_df.shape}")
    
    # Print sample IDs from both datasets for debugging
    print("First few clinical sample columns:", list(clinical_df.columns)[:5])
    print("First few genetic sample columns:", list(gene_data.columns)[:5])
    
    # Convert column names in gene_data to match clinical_df format
    gene_data_renamed = gene_data.copy()
    sample_mapping = {}
    
    # Check if we need to transform sample IDs for matching
    if set(clinical_df.columns).isdisjoint(set(gene_data.columns)):
        print("Sample IDs don't match directly. Attempting to align based on position...")
        # If the number of samples matches, assume they're in the same order
        if len(clinical_df.columns) == len(gene_data.columns):
            sample_mapping = dict(zip(gene_data.columns, clinical_df.columns))
            gene_data_renamed.columns = clinical_df.columns
            print("Aligned samples based on position (same count of samples)")
        else:
            print("Cannot align samples - different counts in clinical and genetic data")
            raise ValueError("Sample counts don't match between clinical and genetic data")
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data_renamed)
    print(f"Linked data shape: {linked_data.shape}")
    is_trait_available = True
    
    # 3. Handle missing values systematically
    print("\nHandling missing values...")
    linked_data = handle_missing_values(linked_data, trait)
    print(f"After handling missing values, data shape: {linked_data.shape}")
    
    # 4. Determine whether the trait and demographic features are biased
    print("\nChecking for bias in features...")
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    
except Exception as e:
    print(f"Error in linking clinical and genetic data: {e}")
    is_trait_available = clinical_df is not None
    linked_data = pd.DataFrame()
    is_biased = True

# 5. Final quality validation
print("\nPerforming final validation...")
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="ATR-X syndrome patients as intellectual disability cases"
)

# 6. Save linked data if usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset not usable for {trait} association studies. Data not saved.")


Normalizing gene symbols...


After normalization, gene data has 20259 genes


Normalized gene data saved to: ../../output/preprocess/Intellectual_Disability/gene_data/GSE100680.csv

Loading clinical data and linking with genetic data...
Loaded clinical data with shape: (2, 34)
First few clinical sample columns: ['GSM2691139', 'GSM2691140', 'GSM2691141', 'GSM2691142', 'GSM2691143']
First few genetic sample columns: ['GSM2691139', 'GSM2691140', 'GSM2691141', 'GSM2691142', 'GSM2691143']
Linked data shape: (34, 20261)

Handling missing values...


After handling missing values, data shape: (34, 20261)

Checking for bias in features...
For the feature 'Intellectual_Disability', the least common label is '1.0' with 16 occurrences. This represents 47.06% of the dataset.
The distribution of the feature 'Intellectual_Disability' in this dataset is fine.

Quartiles for 'Age':
  25%: 45.0
  50% (Median): 65.0
  75%: 65.0
Min: 45.0
Max: 65.0
The distribution of the feature 'Age' in this dataset is fine.


Performing final validation...
A new JSON file was created at: ../../output/preprocess/Intellectual_Disability/cohort_info.json


Linked data saved to ../../output/preprocess/Intellectual_Disability/GSE100680.csv
